In [39]:
import pandas as pd
import numpy as np
import re
import nltk
import tensorflow as tf
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import wordnet
from tensorflow.keras.layers import LSTM, Dense, Embedding
from tensorflow.keras.layers import ReLU
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.models import Model, Sequential
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [2]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
#read data
data = pd.read_csv("/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv")

In [4]:
#size of data
data.shape

(50000, 2)

In [5]:
data.columns

Index(['review', 'sentiment'], dtype='object')

In [6]:
#check null value 
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


In [7]:
data.head(10)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
5,"Probably my all-time favorite movie, a story o...",positive
6,I sure would like to see a resurrection of a u...,positive
7,"This show was an amazing, fresh & innovative i...",negative
8,Encouraged by the positive comments about this...,negative
9,If you like original gut wrenching laughter yo...,positive


In [8]:
data['sent_bin'] = data['sentiment'].replace({'positive': 1, 'negative': 0})

In [10]:
data.head()

,review,sentiment,sent_bin
0,One of the other reviewers has mentioned that ...,positive,1
1,A wonderful little production. <br /><br />The...,positive,1
2,I thought this was a wonderful way to spend ti...,positive,1
3,Basically there's a family where a little boy ...,negative,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,1


In [ ]:
df = data[0:5000]

In [ ]:
df.shape

In [11]:
#filtering the text
def full_form(text):
    text = text.lower()
    plain = re.sub(r'[<>?\.,!"(\)\/[\]]', '', text)
    plain = plain.replace("don't", "do not")
    plain = plain.replace("won't", "will not")
    plain = plain.replace("haven't", "have not")
    plain = plain.replace("can't", "cannot")
    plain = plain.replace("she's", "she is")
    plain = plain.replace("he's", "he is")
    plain = plain.replace("there're", "there are")
    plain = plain.replace("they'd", "they would")
    plain = plain.replace("\'ll", " will")
    return plain              

In [12]:
#initialize the stemmer and lemmentizer
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

In [13]:
#preprocessing  text 
def remove_stop_words(data):
  corpus = []
  for i in range(0, len(data)):
    #split the sentence
    plain = full_form(data['review'][i])
    sentence = plain.split()
    
    #check and stem the word
    review_processed = [stemmer.stem(word) for word in sentence if not word in stopwords.words('english')]
    #rebuild the sentence
    review_joint = ' '.join(review_processed)
    #add the sentence into list
    corpus.append(review_joint)
  return corpus

In [14]:
review_processed = remove_stop_words(data)

In [15]:
y = data['sent_bin']

In [16]:
#vocabular size
voc_size = 5000

In [17]:
#convert into one hot vector
onehot_text = [one_hot(word, voc_size) for word in review_processed]

In [50]:
len(onehot_text)

50000

# Embedding

In [19]:
sent_length = 200

In [20]:
#embedding
embedd_docs = pad_sequences(onehot_text, padding='pre', maxlen=sent_length)

In [21]:
len(review_processed[0].split()), len(onehot_text[0])

(165, 165)

In [22]:
embedd_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0, 4541, 2953, 1096, 1597, 1450,  310, 4728,  520, 3342,
       2165, 3367, 4902, 1409, 2582,  324, 2330,  310, 4946,   20, 4317,
       1412, 4125, 3342, 1980,  366, 3049, 1486,  548, 3032, 1754, 1486,
        723, 1703, 3796, 1061, 2531, 1412, 2326,  267, 2707, 2109, 1409,
       4941,  310, 2487, 3614, 2740,  279,  174, 2587, 2368, 2950, 2398,
       2949,  967, 4436, 4972, 1907,  799,   92,  558, 2158,  170,  415,
         80, 2619, 2348,  967, 4585,  111,  332, 3930, 3557, 4764, 3847,
        827, 3983, 3440, 4163, 4141, 1964, 4561, 1639, 3359, 1745, 3665,
        130, 1409,  614, 2762, 2389, 4738, 1486, 4781, 4683, 2204, 1486,
       4747,   34,    4, 4877,  657,  383, 3042,   34, 2019,   34, 2878,
       1232, 1453, 2582, 4728, 1627, 3086, 2330, 45

In [23]:
#shape of data
len(embedd_docs), y.shape

(50000, (50000,))

In [25]:
#convert into numpy array
X_data = np.array(embedd_docs)
y_data = np.array(y)
#y_data = np.squeeze(y_data)

In [26]:
X_data.shape, y_data.shape

((50000, 200), (50000,))

In [27]:
#split dataset for training and testing
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.3, random_state=42)

In [28]:
print(f'training: {X_train.shape} - {y_train.shape}')
print(f'testing: {X_test.shape} - {y_test.shape}')

training: (35000, 200) - (35000,)
testing: (15000, 200) - (15000,)


In [30]:
y_train[0:10]

array([0, 1, 0, 1, 0, 0, 0, 0, 1, 0])

# Model

In [24]:
## Creating model
embedding_vector_features=100
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 100)          500000    
                                                                 
 lstm (LSTM)                 (None, 100)               80400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 580,501
Trainable params: 580,501
Non-trainable params: 0
_________________________________________________________________
None


In [31]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64)

Epoch 1/10
547/547 [==============================] - 69s 112ms/step - loss: 0.3963 - accuracy: 0.8191 - val_loss: 0.3565 - val_accuracy: 0.8464
Epoch 2/10
547/547 [==============================] - 24s 44ms/step - loss: 0.2813 - accuracy: 0.8852 - val_loss: 0.3279 - val_accuracy: 0.8581
Epoch 3/10
547/547 [==============================] - 14s 26ms/step - loss: 0.2441 - accuracy: 0.9026 - val_loss: 0.3638 - val_accuracy: 0.8373
Epoch 4/10
547/547 [==============================] - 13s 24ms/step - loss: 0.2058 - accuracy: 0.9193 - val_loss: 0.3620 - val_accuracy: 0.8479
Epoch 5/10
547/547 [==============================] - 10s 19ms/step - loss: 0.1786 - accuracy: 0.9311 - val_loss: 0.4206 - val_accuracy: 0.8509
Epoch 6/10
547/547 [==============================] - 10s 18ms/step - loss: 0.1519 - accuracy: 0.9415 - val_loss: 0.4305 - val_accuracy: 0.8486
Epoch 7/10
547/547 [==============================] - 11s 19ms/step - loss: 0.1274 - accuracy: 0.9533 - val_loss: 0.4994 - val_accuracy

# Model Evaluation

In [32]:
model.evaluate(X_test, y_test)

469/469 [==============================] - 3s 5ms/step - loss: 0.6182 - accuracy: 0.8349


[0.6181602478027344, 0.8348666429519653]

In [35]:
predictions = (model.predict(X_test) > 0.5).astype("int32")

469/469 [==============================] - 3s 5ms/step


In [38]:
#confusion matrix 
confusion_matrix(y_test,predictions)

array([[5997, 1414],
       [1063, 6526]])

In [41]:
accuracy_score(y_test,predictions)

0.8348666666666666

# Testing model with text

In [43]:
print(data['review'][0],'\n')
print(data['sentiment'][0])

One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fac

In [55]:
text = data[0:1]
#preprocessing 
processed_text = remove_stop_words(text)
#one hot encoding
onehot_sent = [one_hot(word, voc_size) for word in processed_text]
#embedding
embedd_docs = pad_sequences(onehot_sent, padding='pre', maxlen=sent_length)
#convert into numpy array
X_sample = np.array(embedd_docs)

In [58]:
sample_prediction = (model.predict(X_sample) > 0.5).astype("int32")

1/1 [==============================] - 0s 22ms/step


In [59]:
sample_prediction

array([[1]], dtype=int32)

# Save the model

In [60]:
model.save("movie-senti-analysis.h5")